In [27]:
!pip install wptools

In [28]:
!pip install deep-translator

In [29]:
!pip install beautifulsoup4

## Mount Google Drive

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'
# %cd 'Thesis-MSc/wikipedia-scraping/'

[Errno 2] No such file or directory: 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'
/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [32]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [33]:
page = wptools.page('September_1982_Lebanese_presidential_election')
page.get_parse()

en.wikipedia.org (parse) September_1982_Lebanese_presidential_election
September 1982 Lebanese presidential election (en) data
{
  infobox: <dict(19)> election_name, country, type, ongoing, previ...
  pageid: 70989313
  parsetree: <str(9583)> <root><template><title>Infobox election</...
  requests: <list(1)> parse
  title: September 1982 Lebanese presidential election
  wikibase: Q112724813
  wikidata_url: https://www.wikidata.org/wiki/Q112724813
  wikitext: <str(6929)> {{Infobox election| election_name = Septem...
}


In [34]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Bachir_Gemayel",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('September 1982 Lebanese presidential election')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1926 Lebanese presidential election
1929 Lebanese presidential election
1936 Lebanese presidential election
1943 Lebanese presidential election
1948 Lebanese presidential election
1952 Lebanese presidential election
1958 Lebanese presidential election
1964 Lebanese presidential election
1970 Lebanese presidential election
1976 Lebanese presidential election
1982 Lebanon War
1998 Lebanese presidential election
2008 Lebanese presidential election
2014–2016 Lebanese presidential election
2022–2024 Lebanese presidential election
24 November 1989 Lebanese presidential election
5 November 1989 Lebanese presidential election
Achrafieh
Ain El Remmaneh
Al-Tanzim
Al Jazeera Media Network
Alfred Naqqache
Amine Gemayel
Arabic language
Ariel Sharon
As-Saiqa
Assassination of Bachir Gemayel
August 1982 Lebanese presidential election
Ayoub Tabet
Battle of Zahleh
Battle of the Hotels
Bechara El Khoury
Beirut
Beqaa Governorate
Bikfaya
Black Saturday (Lebanon)
Bsharri
Camille Chamoun
Chamoun famil

In [35]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [36]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [37]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [38]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

['Al-Tanzim']
-----------------
name:  ['Al-Tanzim']
founders:  
ideology:  ['Anti-communism', 'Ultranationalism', 'Lebanese nationalism']
position:  -
START
Cannot find the link: http://en.wikipedia.org/wiki/AlTanzim
name:  Al-Tanzim
name ar:  التنظيم (مترجمه)
name ar link:  -1
END
START
ideology:  Anti-communism
ideology ar:  معاداة الشيوعية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B9%D8%A7%D8%AF%D8%A7%D8%A9_%D8%A7%D9%84%D8%B4%D9%8A%D9%88%D8%B9%D9%8A%D8%A9
ideology:  Ultranationalism
ideology ar:  عصبية قومية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D8%B5%D8%A8%D9%8A%D8%A9_%D9%82%D9%88%D9%85%D9%8A%D8%A9
ideology:  Lebanese nationalism
ideology ar:  قومية لبنانية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
START
END


en.wikipedia.org (parse) Al Jazeera Media Network
en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:Participation of Catherine Asht...
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon official portrait ...
en.wikipedia.org (parse) As-Saiqa


['As-Saiqa']
-----------------
name:  ['As-Saiqa']
founders:  
ideology:  ['Baathism', 'Pan-Arabism']
position:  ['Far-left']
START
Cannot find the link: http://en.wikipedia.org/wiki/AsSaiqa
name:  As-Saiqa
name ar:  كما الصاعقة (مترجمه)
name ar link:  -1
END
START
ideology:  Baathism
ideology ar:  بعثية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D8%B9%D8%AB%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/PanArabism
ideology:  Pan-Arabism
ideology ar:  القومية العربية (مترجمه)
ideo ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Farleft
position:  Far-left
position ar:  أقصى اليسار (مترجمه)
position ar link:  -1
END
START
END


en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:The Kataeb Headquarters after E...
en.wikipedia.org (parse) August 1982 Lebanese presidential election
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:1922 Ayoub Tabet.jpg
en.wikipedia.org (parse) Battle of Zahleh
en.wikipedia.org (imageinfo) File:Zahle,Lebanon.JPG
en.wikipedia.org (parse) Battle of the Hotels
en.wikipedia.org (imageinfo) File:Holiday Inn Beirut 4221308304 3...
en.wikipedia.org (parse) Bechara El Khoury
en.wikipedia.org (imageinfo) File:Bechara El Khoury - 1947.jpg
en.wikipedia.org (parse) Beirut
en.wikipedia.org (parse) Beqaa Governorate
en.wikipedia.org (parse) Bikfaya
en.wikipedia.org (parse) Black Saturday (Lebanon)
en.wikipedia.org (parse) Bsharri
en.wikipedia.org (parse) Camille Chamoun
en.wikipedia.org (imageinfo) File:Camille chamoun.jpg
en.wikipedia.org (parse) Chamoun family
en.wikipedia.org (parse) Charles Debbas
en.wikipedia.org (imageinfo) File

...

[[Mahmoud Abbas]]
['Palestine Liberation Organization']
-----------------
name:  ['Palestine Liberation Organization']
founders:  Mahmoud Abbas,
ideology:  ['August 2024 * Baathism * Marxism', 'Palestinian nationalism', 'Anti-Zionism', 'Secularism', 'August 2024 * Factions:', 'Pan-Arabism', 'August 2024', 'Baathism', 'One-state solution', 'Arab nationalism', 'Anti-imperialism', 'cn', 'ubl', 'One-state solution * Anti-Zionism', 'Marxism', 'Factions:', 'Arab socialism']
position:  ['Left-wing']
START
name:  Palestine Liberation Organization
name ar:  منظمة التحرير الفلسطينية 
name ar link:  https://ar.wikipedia.org/wiki/%D9%85%D9%86%D8%B8%D9%85%D8%A9_%D8%A7%D9%84%D8%AA%D8%AD%D8%B1%D9%8A%D8%B1_%D8%A7%D9%84%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D8%A9
END
START
Cannot find the link: http://en.wikipedia.org/wiki/August   Baathism  Marxism
ideology:  August 2024 * Baathism * Marxism
ideology ar:  أغسطس البعثية الماركسية (مترجمه)
ideo ar link:  -1
ideology:  Palestinian nationalism
ideolog

en.wikipedia.org (parse) Pan-arabism
en.wikipedia.org (parse) Paris
en.wikipedia.org (parse) Parliament of Lebanon
en.wikipedia.org (parse) Patronymic
en.wikipedia.org (parse) Petro Trad
en.wikipedia.org (imageinfo) File:Petro Trad.jpg
en.wikipedia.org (parse) Philip Habib
en.wikipedia.org (imageinfo) File:Philip Habib.jpg
en.wikipedia.org (parse) Philippe Pétain
en.wikipedia.org (imageinfo) File:Pétain - portrait photographique.jpg
en.wikipedia.org (parse) Pierre-Georges Arlabosse
en.wikipedia.org (parse) Pierre Amine Gemayel
en.wikipedia.org (imageinfo) File:Pierre Amine Gemayel.jpg
en.wikipedia.org (parse) Pierre Gemayel
en.wikipedia.org (imageinfo) File:Pierre-Gemayel.jpg
en.wikipedia.org (parse) President of Lebanon
en.wikipedia.org (parse) Privat-Antoine Aubouard
en.wikipedia.org (parse) René Moawad
en.wikipedia.org (imageinfo) File:René Moawad.jpg
en.wikipedia.org (parse) Romanization of Arabic
en.wikipedia.org (parse) Ronald Reagan
en.wikipedia.org (imageinfo) File:Official Por

Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Economic populism', 'Fascism', 'Syrian irredentism', 'collapsible list', 'Antisemitism', 'Anti-communism', 'ubl', 'Syrian nationalism', 'Social nationalism']
position:  ['sometimes being labeled far-right.', 'Syncretic efn', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'and right', 'and right', 'sometimes being labeled far-right.']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Economic populism
ideology ar:  قومية اقتصادية 
ideo ar link:  https://ar.wikipedia.org/wiki

en.wikipedia.org (parse) Tabrieh
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Texas
en.wikipedia.org (parse) The Center for American and International Law
en.wikipedia.org (parse) Tigers Militia (Lebanon)
en.wikipedia.org (parse) Tigers Resistance
en.wikipedia.org (parse) Tony Frangieh
en.wikipedia.org (imageinfo) File:Tony Frangieh.jpg
en.wikipedia.org (parse) United Nations Security Council
en.wikipedia.org (parse) United Nations Security Council Resolution 520
en.wikipedia.org (imageinfo) File:Troepen idf.jpg
en.wikipedia.org (parse) United States
en.wikipedia.org (parse) Wayback Machine
en.wikipedia.org (parse) William Hawi
en.wikipedia.org (imageinfo

skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Reliable sources
skipping Wikipedia:UGC
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template talk:Maronite politics sidebar
skipping Template talk:Presidents of Lebanon
skipping Help:Authority control
skipping Help:IPA/Lebanese Arabic
skipping Category:Accuracy disputes from July 2024
skipping Category:Articles lacking reliable references from July 2024
skipping Category:Articles with dead external links from July 2024
skipping Category:Articles with incomplete citations from July 2024
skipping Category:Articles with self-published sources from July 2024
skipping Category:Articles with unsourced statements from July 2018
skipping Category:Use dmy dates from December 2013
skipping Category:Wikipedia

In [39]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Al-Tanzim,"Anti-communism,Ultranationalism,Lebanese natio...",-,
1,As-Saiqa,"Baathism,Pan-Arabism",Far-left,
2,Constitutional Bloc,"Pro-National Pact,Nonsectarianism,Secular libe...",Centre,
3,Free Patriotic Movement,"Lebanese nationalism,Christian democracy,Civic...","ubl,Centre,centre-left,Right-wing",
4,Guardians of the Cedars,"Anti-Palestinianism<br/ >Secularism,Fascism,An...",Far-right,
5,Lebanese Kataeb Party,"Christian nationalism,Lebanese nationalism,Fal...","Right-wing,far-right,Centre-right","Samy Gemayel,Pierre Gemayel,"
6,Lebanese Forces,"Liberal conservatism,Christian democracy,Leban...",Right-wing,"Samir Geagea,Bachir Gemayel,Vice-president,"
7,Lebanese Forces,"Christian nationalism,Federalism in Lebanon,An...",Right-wing to far-right,
8,Lebanese Forces,"Christian nationalism,Federalism in Lebanon,An...",Right-wing to far-right,
9,Lebanese Forces,"Christian nationalism,Federalism in Lebanon,An...",Right-wing to far-right,


In [40]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,التنظيم (مترجمه),"معاداة الشيوعية ,عصبية قومية ,قومية لبنانية",-,
1,كما الصاعقة (مترجمه),"بعثية ,القومية العربية (مترجمه)",أقصى اليسار (مترجمه),
2,الكتلة الدستورية (مترجمه),"العهد الوطني (مترجمه),اللاطائفية ,ليبرالية علم...",مركز (توضيح),
3,التيار الوطني الحر,"قومية لبنانية ,ديمقراطية مسيحية ,قومية مدنية ,...","أوبل (مترجمه),مركز (توضيح) ,يسار الوسط (مترجمه...",
4,حراس الأرز,"معاداة فلسطين والعلمانية (مترجمه),فاشية ,قومية...",اليمين المتطرف (مترجمه),
5,حزب الكتائب اللبنانية (مترجمه),"قومية مسيحية ,قومية لبنانية ,الفلانخية ,معاداة...","يمينية ,أقصى اليمين (مترجمه),يمين الوسط (مترجمه)","سامي الجميل ,بيار الجميل"
6,القوات اللبنانية,"محافظة ليبرالية ,ديمقراطية مسيحية ,قومية لبنانية",يمينية,"سمير جعجع ,بشير الجميل ,نائب الرئيس"
7,القوات اللبنانية,"قومية مسيحية ,الفيدرالية في لبنان (مترجمه),منا...",من اليمين إلى أقصى اليمين (مترجمه),
8,القوات اللبنانية,"قومية مسيحية ,الفيدرالية في لبنان (مترجمه),منا...",من اليمين إلى أقصى اليمين (مترجمه),
9,القوات اللبنانية,"قومية مسيحية ,الفيدرالية في لبنان (مترجمه),منا...",من اليمين إلى أقصى اليمين (مترجمه),


In [41]:
datasets_dir = 'datasets_updated/September_1982_Lebanese_presidential_election/'

In [42]:
mkdir(datasets_dir)
df_political_party_en.to_excel(os.path.join(datasets_dir, 'political_parties_en.xlsx'), index=False)
df_political_party_ar.to_excel(os.path.join(datasets_dir, 'political_parties_ar.xlsx'), index=False)

In [43]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [44]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

['Alfred Georges Naqqache']
name_inserted before: ['Alfred Georges Naqqache']
name_inserted after: Alfred Georges Naqqache
-----------------
name:  ['Alfred Georges Naqqache']
native_name:  ['nobold', 'ألفرد جورج النقاش']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  -
name ar:  ألفرد نقاش 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
name ar:  ألفرد نقاش 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  -


en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:Participation of Catherine Asht...


['Amine Gemayel']
name_inserted before: ['Amine Gemayel']
name_inserted after: Amine Gemayel
-----------------
name:  ['Amine Gemayel']
native_name:  ['nobold', 'أمين الجميٌل']
nickname:  -
religion:  -
party:  ['Kataeb Party']
nationality:  -
name ar:  أمين الجميل 
START
political party:  Kataeb Party
political party ar:  حزب الكتائب اللبنانية 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D9%83%D8%AA%D8%A7%D8%A6%D8%A8_%D8%A7%D9%84%D9%84%D8%A8%D9%86%D8%A7%D9%86%D9%8A%D8%A9
END
name ar:  أمين الجميل 
religion ar:  ['حزب الكتائب اللبنانية ']
party ar:  ['Kataeb Party']
nationality ar:  -


en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon official portrait ...


...

['Élias Sarkis']
name_inserted before: ['Élias Sarkis']
name_inserted after: Élias Sarkis
-----------------
name:  ['Élias Sarkis']
native_name:  ['إلياس سركيس', 'nobold']
nickname:  -
religion:  -
party:  ['Chehabist']
nationality:  ['Lebanese']
name ar:  إلياس سركيس 
START
political party:  Chehabist
political party ar:  الشهابية (أيديولوجية) 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%B4%D9%87%D8%A7%D8%A8%D9%8A%D8%A9_(%D8%A3%D9%8A%D8%AF%D9%8A%D9%88%D9%84%D9%88%D8%AC%D9%8A%D8%A9)
END
START
nationality:  Lebanese
nationality ar:  لبناني (مترجمه)
nationality ar link:  -1
END
name ar:  إلياس سركيس 
religion ar:  ['الشهابية (أيديولوجية) ']
party ar:  ['Chehabist']
nationality ar:  ['لبناني (مترجمه)']


en.wikipedia.org (parse) Émile Eddé
en.wikipedia.org (imageinfo) File:Émile Eddé Official portrait.jpg


['Émile Eddé']
name_inserted before: ['Émile Eddé']
name_inserted after: Émile Eddé
-----------------
name:  ['Émile Eddé']
native_name:  ['nobold', 'إميل إدّه']
nickname:  -
religion:  -
party:  ['National Bloc']
nationality:  -
name ar:  إميل إده 
START
political party:  National Bloc
political party ar:  الكتلة الوطنية (توضيح) 
political party ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D9%83%D8%AA%D9%84%D8%A9_%D8%A7%D9%84%D9%88%D8%B7%D9%86%D9%8A%D8%A9_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
name ar:  إميل إده 
religion ar:  ['الكتلة الوطنية (توضيح) ']
party ar:  ['National Bloc']
nationality ar:  -


en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg


['Émile Lahoud']
name_inserted before: ['Émile Lahoud']
name_inserted after: Émile Lahoud
-----------------
name:  ['Émile Lahoud']
native_name:  ['اميل لحود', 'nobold']
nickname:  -
religion:  -
party:  ['Independent']
nationality:  -
name ar:  إميل لحود 
START
political party:  Independent
political party ar:  مستقل (توضيح) 
political party ar link:  https://ar.wikipedia.org/wiki/%D9%85%D8%B3%D8%AA%D9%82%D9%84_(%D8%AA%D9%88%D8%B6%D9%8A%D8%AD)
END
name ar:  إميل لحود 
religion ar:  ['مستقل (توضيح) ']
party ar:  ['Independent']
nationality ar:  -
skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Reliable sources
skipping Wikipedia:UGC
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template talk:Maronite politics sidebar
skipping Template talk:Preside

In [45]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Alfred Georges Naqqache,Kataeb Party,-,-
1,Amine Gemayel,Kataeb Party,-,-
2,nobold,plainlist,-,-
3,Ayoub Tabet,-,-,-
4,Bechara Khalil El Khoury,Constitutional Bloc,-,-
5,Camille Chamoun,"National Liberal Party,Constitutional Bloc",-,-
6,nobold,-,-,-
7,Charles Helou,-,Lebanese,-
8,Dany Chamoun,National Liberal Party,-,-
9,Elias Hrawi,Independent,Lebanese,-


In [46]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,-,حزب الكتائب اللبنانية,ألفرد نقاش
1,-,-,حزب الكتائب اللبنانية,أمين الجميل
2,-,-,قائمة عادية (مترجمه),أرئيل شارون
3,-,-,-,-1
4,-,-,الكتلة الدستورية (مترجمه),بشارة الخوري
5,-,-,"الحزب الليبرالي الوطني (مترجمه),الكتلة الدستور...",كميل شمعون
6,-,-,-,شارل دباس
7,-,لبناني (مترجمه),-,شارل حلو
8,-,-,الحزب الليبرالي الوطني (مترجمه),داني شمعون
9,-,لبناني (مترجمه),مستقل (توضيح),إلياس الهراوي


In [47]:
mkdir(datasets_dir)
df_politician_en.to_excel(os.path.join(datasets_dir, 'politicians_en.xlsx'), index=False)
df_politician_ar.to_excel(os.path.join(datasets_dir, 'politicians_ar.xlsx'), index=False)

In [48]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [49]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1926 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1929 Lebanese presidential election
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) 1936 Lebanese presidential election
en.wikipedia.org (parse) 1943 Lebanese presidential election
en.wikipedia.org (parse) 1948 Lebanese pre

-----------------
name:  -
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'flagcountry']
skipping Achrafieh because name is not found


en.wikipedia.org (parse) Ain El Remmaneh
en.wikipedia.org (parse) Al-Tanzim
en.wikipedia.org (parse) Al Jazeera Media Network
en.wikipedia.org (parse) Alfred Naqqache
en.wikipedia.org (imageinfo) File:Alfred Naqqache 2.jpg
en.wikipedia.org (parse) Amine Gemayel
en.wikipedia.org (imageinfo) File:Participation of Catherine Asht...
en.wikipedia.org (parse) Arabic language
en.wikipedia.org (imageinfo) File:Arabic albayancalligraphy.svg
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon official portrait ...
en.wikipedia.org (parse) As-Saiqa
en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:The Kataeb Headquarters after E...
en.wikipedia.org (parse) August 1982 Lebanese presidential election
en.wikipedia.org (parse) Ayoub Tabet
en.wikipedia.org (imageinfo) File:1922 Ayoub Tabet.jpg
en.wikipedia.org (parse) Battle of Zahleh
en.wikipedia.org (imageinfo) File:Zahle,Lebanon.JPG
en.wikipedia.org (parse) Battle of the Hote

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Beqaa Governorate


...

-----------------
name:  United States
subdivision_type:  -
subdivision_name:  -
name ar:  الولايات المتحدة 


en.wikipedia.org (parse) Wayback Machine
en.wikipedia.org (parse) William Hawi
en.wikipedia.org (imageinfo) File:William Amine Hawi.jpg
en.wikipedia.org (parse) Yasser Arafat
en.wikipedia.org (imageinfo) File:Leader of the PLO, Yasser Arafa...
en.wikipedia.org (parse) YouTube video (identifier)
en.wikipedia.org (parse) Zahlé


-----------------
name:  Zahlé
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon', 'Flag']
name ar:  زحلة 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
subdivision_name:  Flag
subdivision_name ar:  علم (راية) 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D8%B9%D9%84%D9%85_(%D8%B1%D8%A7%D9%8A%D8%A9)
END


en.wikipedia.org (parse) Ze'ev Jabotinsky
en.wikipedia.org (imageinfo) File:זאב ז'בוטינסקי-JNF010760.jpeg
en.wikipedia.org (parse) Élias Sarkis
en.wikipedia.org (imageinfo) File:Elias Youssef Sarkis, President...
en.wikipedia.org (parse) Émile Eddé
en.wikipedia.org (imageinfo) File:Émile Eddé Official portrait.jpg
en.wikipedia.org (parse) Émile Lahoud
en.wikipedia.org (imageinfo) File:Lahoud in Brasil 1 (cropped).jpg


skipping Wikipedia:Citation needed
skipping Wikipedia:Citing sources
skipping Wikipedia:Link rot
skipping Wikipedia:Protection policy
skipping Wikipedia:Reliable sources
skipping Wikipedia:UGC
skipping Wikipedia:Verifiability
skipping Template:Maronite politics sidebar
skipping Template:Marriage
skipping Template:Marriage/doc
skipping Template:Presidents of Lebanon
skipping Template talk:Maronite politics sidebar
skipping Template talk:Presidents of Lebanon
skipping Help:Authority control
skipping Help:IPA/Lebanese Arabic
skipping Category:Accuracy disputes from July 2024
skipping Category:Articles lacking reliable references from July 2024
skipping Category:Articles with dead external links from July 2024
skipping Category:Articles with incomplete citations from July 2024
skipping Category:Articles with self-published sources from July 2024
skipping Category:Articles with unsourced statements from July 2018
skipping Category:Use dmy dates from December 2013
skipping Category:Wikipedia

In [50]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,-,Country,"Lebanon,flagcountry"
1,Beirut,Country,Lebanon
2,-,Country,Lebanon
3,Bikfaya,Country,"Lebanon,flagcountry"
4,Bsharri,Country,LBN
5,-,Country,United States
6,Damour,Country,LBN
7,Ehden,Country,"Lebanon,flagcountry"
8,Israel,-,-
9,Keserwan District,Country,"Lebanon,Flag"


In [51]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,لبنان,بلد,بيروت
1,"لبنان ,علم الدولة (مترجمه)",بلد,بكفيا
2,لبنان (مترجمه),بلد,بشري (لبنان)
3,لبنان (مترجمه),بلد,الدامور
4,"لبنان ,علم الدولة (مترجمه)",بلد,إهدن
5,-,-,إسرائيل
6,"لبنان ,علم (راية)",بلد,قضاء كسروان
7,-,-,لبنان
8,"لبنان ,علم (راية)",بلد,قضاء المتن
9,الاستخبارات والمراقبة والرصد (مترجمه),بلد,نهاريا


In [52]:
mkdir(datasets_dir)
df_locations_en.to_excel(os.path.join(datasets_dir, 'locations_en.xlsx'), index=False)
df_locations_ar.to_excel(os.path.join(datasets_dir, 'locations_ar.xlsx'), index=False)